In [6]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
import pandas as pd

In [ ]:
model = TypePredictor()
model.fit(number=1000)

In [ ]:
dataset = SimpleDatasetGenerator()

df, y = dataset.generate_simple_dataframe()

model.predict_dataframe(df)

In [4]:
df

,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex
0,ZRDCCT39S41F696T,02711130738,Via S. Vincenzo,88047,/,NaN,TOSCANA,SAN LUCA,2008.0,6464.0,...,palestina,BG,jarvas,woodcock,lo stomaco,layla.springe@cox.net,+39-351-255-531118,"28.807,00 €",1967-03-16,M
1,MSSYNS46C22E953S,06043410825,Via Pian Due Torri,64020.0,\n,AR,MARCHE,MASULLAS,1994.0,-81087.0,...,irlanda,PH,ilvi,nietupski,il fratello,mari_lueckenbach@yahoo.com,+39-313-555-0011,"85.245,00 €",NaN,donna
2,CLMPGR86M24E170R,NaN,Via Delle Piscine,85052,#RIF!,BL,PUGLIA,GINOSA,1997.0,34804.0,...,thailandia,ZM,bhrigu,fausnaughtjr,senza,brandon_callaro@hotmail.com,+39-328-555-868,"33.784,00 €",1992-07-14,M
3,RMNPMR30B03I800C,04320521000,Via Madonna Del Piano,91023,,SV,FRIULI VENEZIA GIULIA,CONTIGLIANO,2009.0,NaN,...,sierra leone,SI,domini,erler,NaN,denise@patak.org,NaN,"70.401,00 €",NaN,F
4,MNFMRA30B07M169X,02118360714,Via Prenestina Nuova Km 69,40138,NaN,NaN,FRIULI VENEZIA GIULIA,SAN NICOLO' D'ARCIDANO,2015.0,23984.0,...,siria,YE,cleante,ariyoshi,la nuvola,wilda@cox.net,+39-333-555-9152,"70.759,00 €",1985-05-17,donna
5,NaN,02416740716,Via Paradisa,191,NaN,NaN,MARCHE,PIANORO,2013.0,96454.0,...,micronesia,NP,john,herschaft,non fumerò mai,nan@koppinger.com,+39-330-555-4414,"53.546,00 €",1921-10-29,maschio
6,03104440759,CRRBRN00R02I571Q,Via De' Brozzi,20151.0,#RIF!,FG,NaN,SALI VERCELLESE,1996.0,36193.0,...,figi,BG,paraskeve,grubbs,NaN,oshealy@hotmail.com,+39-350-555-572032,NaN,1927-11-27,femmina
7,01361050352,RZZRNN88T41A149U,Via A. Guiglieri,53034,0,KR,TOSCANA,VALLEDORIA,1997.0,15402.0,...,andorra,BY,fazile,beiswenger,oggi,roosevelt.hoffis@aol.com,+39-366-555-4983,"43.777,00 €",1944-01-18,maschio
8,02189060136,MLVLIU95L70H218D,Via Olivieri,176,/,TE,CAMPANIA,ADELFIA,2023.0,34718.0,...,capo verde,MX,aquista,vanputten,la bibita gassata,lynelle_auber@gmail.com,+39-371-055-504980,"19.797,00 €",1964-02-10,M
9,PLCMND81D51H713O,04523690750,Via S. M. Mazzarello,10063.0,#RIF!,NO,LIGURIA,CALASCA CASTIGLIONE,2009.0,-96396.0,...,guatemala,KI,curtis joseph,kutz,uccelli,ezekiel@chui.com,+39-385-555-080904,"75.637,00 €",1955-10-01,maschio
